In [3]:
!wget https://raw.githubusercontent.com/gohdong/2021_autumn/master/BigData/project1/facebook_combined.txt

--2021-10-01 13:53:24--  https://raw.githubusercontent.com/gohdong/2021_autumn/master/BigData/project1/facebook_combined.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 854362 (834K) [text/plain]
Saving to: ‘facebook_combined.txt’

facebook_combined.t 100%[===================>] 834.34K  --.-KB/s    in 0.008s  

2021-10-01 13:53:25 (107 MB/s) - ‘facebook_combined.txt’ saved [854362/854362]



In [ ]:
!ls

In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz


--2021-10-01 13:54:48--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224445805 (214M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop2.7.tgz’

spark-3.1.2-bin-had 100%[===================>] 214.05M   176MB/s    in 1.2s    

2021-10-01 13:54:50 (176 MB/s) - ‘spark-3.1.2-bin-hadoop2.7.tgz’ saved [224445805/224445805]



In [5]:
!pip install findspark
!pip install pyspark


     |████████████████████████████████| 212.4 MB 58 kB/s 
     |████████████████████████████████| 198 kB 57.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=82f60fb8a46642ec30706b2b14e491d3995ea5b65f8c7ae21f7f30b22918a9d3
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext("local","bruted_force")

In [10]:
input_rdd = sc.textFile("facebook_combined.txt").flatMap(lambda x : [(x.split()[0],x.split()[1]),(x.split()[1],x.split()[0])])

In [11]:
friend_list = input_rdd.groupByKey().mapValues(list)

In [45]:
friend_list.take(1)

[('0',
  ['1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   '13',
   '14',
   '15',
   '16',
   '17',
   '18',
   '19',
   '20',
   '21',
   '22',
   '23',
   '24',
   '25',
   '26',
   '27',
   '28',
   '29',
   '30',
   '31',
   '32',
   '33',
   '34',
   '35',
   '36',
   '37',
   '38',
   '39',
   '40',
   '41',
   '42',
   '43',
   '44',
   '45',
   '46',
   '47',
   '48',
   '49',
   '50',
   '51',
   '52',
   '53',
   '54',
   '55',
   '56',
   '57',
   '58',
   '59',
   '60',
   '61',
   '62',
   '63',
   '64',
   '65',
   '66',
   '67',
   '68',
   '69',
   '70',
   '71',
   '72',
   '73',
   '74',
   '75',
   '76',
   '77',
   '78',
   '79',
   '80',
   '81',
   '82',
   '83',
   '84',
   '85',
   '86',
   '87',
   '88',
   '89',
   '90',
   '91',
   '92',
   '93',
   '94',
   '95',
   '96',
   '97',
   '98',
   '99',
   '100',
   '101',
   '102',
   '103',
   '104',
   '105',
   '106',
   '107',
   '108',
   '109',
   '110',
 

In [12]:
friend_length = sc.broadcast(input_rdd.groupByKey().mapValues(len).collect())

In [13]:
friend_length.value[0]

('0', 347)

In [14]:
def generate_keyfair(x):
	temp = []
	for i in range(len(x)):
		for j in range(i+1,len(x)):
			temp.append(((x[i],x[j]),1))
			# print(((x[i],x[j]),1))
	return temp

In [15]:
key_fair = friend_list.flatMap(lambda x:generate_keyfair(x[1]))

In [44]:
key_fair.take(1)

[(('1', '2'), 1)]

In [29]:
thresh_hold = 0.5

In [30]:
pass_threshhold = key_fair.reduceByKey(lambda x,y : x+y).filter(lambda x: x[1] >= (thresh_hold)*(friend_length.value[int(x[0][0])][1]+friend_length.value[int(x[0][1])][1])/(1+thresh_hold))

In [31]:
temp = pass_threshhold.collect()

In [32]:
temp[0]

(('2', '14'), 9)

In [39]:
a = pass_threshhold.flatMap(lambda x : [((x[0][0]),x[0][1]),((x[0][1]),x[0][0])])

In [40]:
b = a.groupByKey().mapValues(list)

In [41]:
b.take(1)

[('2', ['14', '149', '162', '226'])]

In [43]:
b.collect()

[('2', ['14', '149', '162', '226']),
 ('14',
  ['2', '28', '41', '115', '116', '140', '144', '149', '226', '326', '343']),
 ('149', ['2', '14', '20', '116', '226', '326', '343']),
 ('162', ['2', '226']),
 ('226',
  ['2', '14', '20', '28', '115', '116', '144', '149', '162', '326', '343']),
 ('4', ['78', '181', '195', '218', '273', '275', '306', '328']),
 ('78', ['4', '181', '195', '218', '273', '275', '306', '328']),
 ('181', ['4', '78', '195', '218', '273', '275', '306', '328']),
 ('195', ['4', '78', '181', '218', '273', '275', '306', '328']),
 ('218', ['4', '78', '181', '195', '273', '275', '306', '328']),
 ('273', ['4', '78', '181', '195', '218', '275', '306', '328']),
 ('275', ['4', '78', '181', '195', '218', '273', '306', '328']),
 ('306', ['4', '78', '181', '195', '218', '273', '275', '328']),
 ('328', ['4', '78', '181', '195', '218', '273', '275', '306']),
 ('6', ['89', '95', '147', '219', '319', '327']),
 ('89', ['6', '95', '147', '319']),
 ('95', ['6', '89', '147', '219', '319'